# Working with hyperspectral images

In this class we will learn briefly how to work with hyperspectral images in Python.

## Load images from MATLAB data files

If a hyperspectral image is stored as MATLAB data file (`.mat`) it can be loaded using package `scipy` which we have already installed in one of the previous classes.

In folder `dataset` there is a file `emulsion.mat` with a famous Raman spectral image of oil in water emulsion. It is saved as array with name `X`. Let's load it: 

In [ ]:
# import function which lets load MATLAB data
from scipy.io import loadmat

# read the file and save the result to dictionary "d"
d = loadmat('datasets/emulsion.mat')

# check which elements this dictionary contains
d.keys()

As you can see, indeed the dictionary contains object with name `X`. Let's extract the object and check its size assuming that it is a NumPy array:

In [ ]:
img_arr = d["X"]
img_arr.shape

And it is indeed. As expected, the image has size of 60x60 pixels and 253 channels with Raman intensities at different wavenumbers. Let's show intensity at channel 100 as a pseudocolor image:

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(img_arr[:, :, 100])
plt.title("Channel 100")
plt.colorbar()

It works. If necessary you can reshape the image 3D array to 2D array with rows corresponding to pixels. This let's you work with the image as conventional dataset and e.g. make a line plot with spectra:

In [ ]:
nrows, ncols, nchannels = img_arr.shape
npixels = nrows * ncols

X = img_arr.reshape((npixels, nchannels))

plt.plot(X.T)
plt.xlabel("Spectral channels")
plt.ylabel("Intensity, A.U.")


Let's normalize every spectrum to unit length (L2-norm):

In [ ]:
import numpy as np

def l2norm(x):
    return x / np.sqrt(np.sum(x**2))

X_norm = np.apply_along_axis(l2norm, 1, X)

Let's check the intensity of the spectra to confirm the normalization:

In [ ]:
plt.plot(X_norm.T)
plt.xlabel("Spectral channels")
plt.ylabel("Intensity, A.U.")

Looks like this is not the best preprocessing method for this data, bit let's continue anyway and reshape the new preprocessed spectra back to spectral image and show the intensity map for channel 100 again. We will show both the original data and the preprocessed data:

In [ ]:
img_arr_norm = X_norm.reshape((60, 60, 253))

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(img_arr[:, :, 100])
plt.title("Channel 100 (original)")
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(img_arr_norm[:, :, 100])
plt.title("Channel 100 (normalized)")
plt.colorbar()

I you want to apply various image transformations to the hyperspectral images there are two main options:

1. Use more advanced library Open CV which works with NumPy array directly.
2. Process every channel separately as a PIL grayscale image.

Here is an example how to use the second option to upscale the emulsion image and rotate it by 90 degrees:

In [ ]:
from PIL import Image

# get dimensions of the original image
nrows, ncols, nchannels = img_arr.shape

# compute size of new image
# because we plan to rotate it, rows and cols are swapped
# but in this case it is not needed as they are equal
ncols_new = nrows * 2
nrows_new = ncols * 2

# prepare array for the new image
img_arr_new = np.zeros((nrows_new, ncols_new, nchannels))

# loop to process each channel separately
for i in range(0, nchannels):
    img = Image.fromarray(img_arr[:, :, i])     # get the channel and convert it to PIL Image
    img = img.resize((ncols_new, nrows_new))    # resize the image
    img = img.rotate(90, expand=True)                        # rotate the image by 90 degrees
    img_arr_new[:, :, i] = np.array(img)        # convert it back to NumPy array

plt.imshow(img_arr_new[:, :, 100])
plt.colorbar()


## Working with ENVI images

In order to load hyperspectral images saved in other popular format, ENVI, we will need another library. There are several possibilities in this case:

2. [GDAL](https://gdal.org/index.html) — a translator library for raster and vector geospatial data formats.  
3. A bit outdated, but still functional and more simple package, [Spectral Python](http://www.spectralpython.net) (last release 2/10/2022).

There is also [HypeSpy](https://hyperspy.org/index.html) — a Python framework for exploring, visualizing and analyzing multi-dimensional data, which provides a lot of other possibilities in addition to simply load spectra files.

Below we will try the last simple option to get the hyperspectral data from ENVI format to NumPy array. Let's first install the library:

In [ ]:
! pip install spectral



In the folder `dataset` there is another folder with name `tablets` which contains image of tablets in ENVI format. There are three images: the one of tablet as well as the dark and the white references. Let's load, correct, and visualize it:

In [ ]:
from spectral.io import envi

# open each image by providing path to header file (.hdr) and file with data values (.raw)
dark_ref = envi.open('datasets/tablets/Tablets-darkref.hdr','datasets/tablets/Tablets-darkref.raw')
white_ref = envi.open('datasets/tablets/Tablets-whiteref.hdr', 'datasets/tablets/Tablets-whiteref.raw')
img_ref = envi.open('datasets/tablets/Tablets.hdr','datasets/tablets/Tablets.raw')

# get information about spectral bands
bands = img_ref.bands.centers
bands

In [ ]:
# load the image data and convert to NumPy array

white_img = np.array(white_ref.load())
dark_img = np.array(dark_ref.load())
img = np.array(img_ref.load())

(white_img.shape, dark_img.shape, img.shape)

As you can see, the white and dark references are obtained for 100 lines not for 337 as the original image. What we can stack several images together to meat the required size:

In [ ]:
white_img_full = np.vstack((white_img, white_img, white_img, white_img[:37, :, :]))
white_img_full.shape

In [ ]:
dark_img_full = np.vstack((dark_img, dark_img, dark_img, dark_img[:37, :, :]))
dark_img_full.shape

In fact it was not necessary as in this case we could simply take only first row of the dark and the white reference images, because the images was acquired using [pushbroom scanner](https://en.wikipedia.org/wiki/Push_broom_scanner).

Let's look at the images before correction:

In [ ]:
plt.figure(figsize=(30, 10))

plt.subplot(1, 3, 1)
plt.imshow(dark_img_full[:, :, 100])
plt.colorbar()
plt.title("Dark")

plt.subplot(1, 3, 2)
plt.imshow(white_img_full[:, :, 100])
plt.colorbar()
plt.title("White")

plt.subplot(1, 3, 3)
plt.imshow(img[:, :, 100])
plt.colorbar()
plt.title("Raw image")


Let's compute a difference between the dark and the white images and replace column with zeros by ones to avoid division by zero.

In [ ]:
diff = (white_img_full - dark_img_full)
diff[diff == 0] = 1

Now we can apply correction and look at the corrected image:

In [ ]:
# correct the image by subtracting dark reference and dividing by difference between
# dark and white references
corrected_img = (img - dark_img_full) / diff

# show the image
plt.imshow(corrected_img[:, :, 100])
plt.colorbar()

Let's apply a mask:

In [ ]:
mask = corrected_img[:, :, 100] > 0.4

# ! this will not work
masked_img = corrected_img * mask

The code above is an attempt to process each channel using built in broadcasting option in NumPy, which lets avoid the loops. However it does not work because the dimension we broadcast through is the last one. We can fix this by swap the dimensions (and the return them back):

In [ ]:
masked_img = corrected_img.transpose((2, 0, 1)) * mask
masked_img = masked_img.transpose((1, 2, 0))

In [ ]:

plt.figure(figsize=(30, 10))
plt.subplot(1, 3, 1)
plt.imshow(corrected_img[:, :, 100])
plt.subplot(1, 3, 2)
plt.imshow(mask)
plt.subplot(1, 3, 3)
plt.imshow(masked_img[:, :, 100])

That's all, folks! :)